In [12]:
import json
import numpy as np
import geopandas as gpd
from shapely.geometry import shape
from pyproj import Geod
import tqdm
from scipy.spatial.distance import euclidean
import random
from geopy.distance import geodesic


In [ ]:


class ResourceAnalyzer:
    def __init__(self, geojson_path):
        self.geojson_path = geojson_path
        self.farmlands = self.load_data()

    def load_data(self) -> dict[str, list[gpd.GeoSeries]]:
        """GeoJSONデータを読み込み、FarmerIndicationNumberHashごとに農地をまとめる
        Returns:
            dict: FarmerIndicationNumberHashをキーとし、その農地のリストを値とする辞書
        """
        gdf = gpd.read_file(self.geojson_path)
        farms = {}
        for _, row in gdf.iterrows():
            farmer_id = row['FarmerIndicationNumberHash'] if row['FarmerIndicationNumberHash'] is not None else 'unknown'
            geometry = row['geometry']

            if farmer_id not in farms:
                farms[farmer_id] = []
            farms[farmer_id].append(geometry)
        print(f"Loaded {len(farms)} farmers' farmlands")
        print(farms)
        return farms

    def calculate_area(self, polygon):
        """ポリゴンの面積を計算する（球面積分を使用）"""
        geod = Geod(ellps="WGS84")
        lon, lat = polygon.exterior.coords.xy
        return abs(geod.polygon_area_perimeter(lon, lat)[0]) # 単位は平方メートル


    def calculate_farm_statistics(self, farmer_id):
        """特定の農家の農地の統計情報を計算する"""
        if farmer_id not in self.farmlands:
            print(f"Farmer ID {farmer_id} not found in dataset.")
            return {}
        plots = self.farmlands[farmer_id]
        # 統計情報の計算ロジックをここに追加
        areas = [self.calculate_area(plot) for plot in plots]
        stats = {
            "num_plots": len(plots), # 農地数
            "total_area": sum(areas), # 総面積(m^2)
            "num_large_plots": len([a for a in areas if a > 1000]), # 1000m^2以上の農地数
        }
        return stats

    def estimate_human_resources(self, stats, period_months=12):
        """人的リソースの見積もりを計算する"""
        base_rate = 0.1  # 基本レート（仮定）
        total_area = stats.get("total_area", 0)
        yearly_person_months = total_area * base_rate
        period_person_months = yearly_person_months * (period_months / 12)
        return period_person_months

    def distance_matrix(self, farms):
        """農地間の距離行列を計算する"""
        num = len(farms)
        dist_matrix = np.zeros((num, num))
        centroids = [farm.centroid for farm in farms]
        for i in range(num):
            for j in range(num):
                if i != j:
                    dist_matrix[i, j] = geodesic((centroids[i].y, centroids[i].x), (centroids[j].y, centroids[j].x)).meters
        return dist_matrix

    def greedy_tsp(self, dist_matrix: np.ndarray) -> list[int]:
        """グリーディ法で初期経路を生成"""
        num = len(dist_matrix)
        visited = [False] * num
        path = [0]
        visited[0] = True
        for _ in range(num - 1):
            last = path[-1]
            next_city = np.argmin([dist_matrix[last, j] if not visited[j] else np.inf for j in range(num)])
            path.append(next_city)
            visited[next_city] = True
        return path

    def simulated_annealing(self, dist_matrix, initial_path, temp=1000.0, cooling_rate=0.995, max_iter=1000):
        """焼きなまし法で経路を最適化"""
        def calculate_total_distance(path):
            return sum(dist_matrix[path[i], path[i + 1]] for i in range(len(path) - 1)) + dist_matrix[path[-1], path[0]]

        current_path = initial_path
        current_distance = calculate_total_distance(current_path)
        best_path = list(current_path)
        best_distance = current_distance

        for _ in range(max_iter):
            temp *= cooling_rate
            if temp <= 0.1:
                break

            # 2-opt swap
            if len(current_path) > 2:
                i, j = sorted(random.sample(range(1, len(current_path)), 2))
                new_path = current_path[:i] + current_path[i:j][::-1] + current_path[j:]
                new_distance = calculate_total_distance(new_path)

                if new_distance < current_distance or random.random() < np.exp((current_distance - new_distance) / temp):
                    current_path = new_path
                    current_distance = new_distance

                    if current_distance < best_distance:
                        best_path = list(current_path)
                        best_distance = current_distance
        # print(best_path)

        return best_path, best_distance

    def calculate_travel_distance(self, farms):
        """農地間の移動距離を計算する"""
        dist_matrix = self.distance_matrix(farms)
        initial_path = self.greedy_tsp(dist_matrix)
        best_path, best_distance = self.simulated_annealing(dist_matrix, initial_path)
        return best_distance


    def run_analysis(self, farmer_id):
        """特定の農家に対して全体の分析を実行する"""
        stats = self.calculate_farm_statistics(farmer_id)
        resources = self.estimate_human_resources(stats)
        farms = self.farmlands[farmer_id]
        travel_distance = self.calculate_travel_distance(farms)
        return stats, resources, travel_distance

    def run_total_analysis(self):
        """地域全体のリソースシミュレーションを実行する"""
        total_stats = {"total_area": 0, "num_plots": 0, "num_large_plots": 0}
        total_resources = 0
        total_travel_distance = 0
        for farmer_id in tqdm.tqdm(self.farmlands.keys()):
            stats, resources, travel_distance = self.run_analysis(farmer_id)
            total_stats["num_plots"] += stats.get("num_plots", 0)
            total_stats["total_area"] += stats.get("total_area", 0)
            total_stats["num_large_plots"] += stats.get("num_large_plots", 0)
            total_resources += resources
            total_travel_distance += travel_distance
        return total_stats, total_resources, total_travel_distance


In [ ]:
import math
import geojson
import random
import numpy as np

def distance(p1, p2):
    """2点間のユークリッド距離を計算"""
    return math.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

def total_path_length(points):
    """求めた経路の総距離を計算"""
    return sum(distance(points[i], points[i+1]) for i in range(len(points)-1)) + distance(points[-1], points[0])

def greedy_tsp(points):
    """貪欲法を用いて初期解を生成"""
    unvisited = set(points)
    path = [unvisited.pop()]
    while unvisited:
        next_point = min(unvisited, key=lambda p: distance(path[-1], p))
        path.append(next_point)
        unvisited.remove(next_point)
    return path

def simulated_annealing(points, initial_temp=1000, cooling_rate=0.995, stopping_temp=1e-3):
    """焼きなまし法を用いてTSPの近似解を求める"""
    current_solution = greedy_tsp(points)
    current_distance = total_path_length(current_solution)
    best_solution = list(current_solution)
    best_distance = current_distance
    temp = initial_temp

    while temp > stopping_temp:
        i, j = sorted(random.sample(range(len(points)), 2))
        new_solution = current_solution[:]
        new_solution[i:j] = reversed(new_solution[i:j])
        new_distance = total_path_length(new_solution)

        if new_distance < current_distance or random.random() < math.exp((current_distance - new_distance) / temp):
            current_solution = new_solution
            current_distance = new_distance
            if new_distance < best_distance:
                best_solution = new_solution
                best_distance = new_distance

        temp *= cooling_rate

    return best_solution

# GeoJSONデータの読み込み
def load_geojson(filename):
    with open(filename, 'r') as f:
        data = geojson.load(f)
    return [tuple(feature["geometry"]["coordinates"]) for feature in data["features"]]

# 例: GeoJSONファイルを読み込んでTSP経路を計算
filename = "points.geojson"
points = load_geojson(filename)
optimal_path = simulated_annealing(points)
path_length = total_path_length(optimal_path)

print(f"総移動距離: {path_length}")


In [ ]:
# 農家idが"2dacba93d45b0f46a25b29b985bd90e2"の人に対して分析を実行
before_path = "../../notebooks/data/geojson_filtered_by_settlement/筑地.geojson"
after_path = "../../../データ/test_reorganized.geojson"

before_analyzer = ResourceAnalyzer(before_path)
stats, resources, travel_distance = before_analyzer.run_analysis("2dacba93d45b0f46a25b29b985bd90e2")
print(stats)
print(resources)
print(travel_distance)
print("--------------------")

# 地域全体の分析を実行
total_stats, total_resources, total_travel_distance = before_analyzer.run_total_analysis()
print(total_stats)
print(total_resources)
print(total_travel_distance)


Loaded 116 farmers' farmlands
{'a504f94f9a34e120b11c26dcc561cc93': [<POLYGON ((140.378 36.375, 140.378 36.375, 140.379 36.375, 140.379 36.375, 1...>, <POLYGON ((140.378 36.372, 140.379 36.371, 140.378 36.371, 140.378 36.371, 1...>], '0f1cd9d3f5b6f65893e4f31c96c52045, d0dbbea72d1f340a64f82d1060dc9a42': [<POLYGON ((140.377 36.376, 140.377 36.376, 140.377 36.376, 140.378 36.376, 1...>], '745fb7229d1395c231177495b777fd2e': [<POLYGON ((140.377 36.376, 140.377 36.376, 140.378 36.376, 140.378 36.376, 1...>, <POLYGON ((140.377 36.375, 140.377 36.375, 140.378 36.376, 140.378 36.375, 1...>, <POLYGON ((140.378 36.375, 140.378 36.376, 140.379 36.376, 140.379 36.375, 1...>, <POLYGON ((140.378 36.375, 140.379 36.375, 140.379 36.375, 140.378 36.375, 1...>, <POLYGON ((140.379 36.37, 140.38 36.37, 140.38 36.37, 140.38 36.37, 140.379 ...>], '2e20b8f76a7381f7e923f5f174579807': [<POLYGON ((140.374 36.37, 140.374 36.37, 140.374 36.37, 140.374 36.37, 140.3...>], 'a96dd9b2cfd69e2d52d31a0fee58d768': [<POLYGON

100%|██████████| 116/116 [00:00<00:00, 381.79it/s]

{'total_area': 513551.5166834518, 'num_plots': 348, 'num_large_plots': 217}
51355.15166834516
78906.91047899227


In [ ]:
if __name__ == "__main__":
    # 最適化前のリソースシミュレーション
    before_analyzer = ResourceAnalyzer("../../notebooks/data/geojson_filtered_by_settlement/筑地.geojson")
    before_total_stats, before_total_resources, before_travel_distance = before_analyzer.run_total_analysis()
    print("\n地域全体の農地統計情報:")
    print(json.dumps(before_total_stats, indent=2, ensure_ascii=False))
    print("\n地域全体の必要な人的リソース:")
    print(json.dumps(before_total_resources, indent=2, ensure_ascii=False))
    print("\n地域全体の移動距離:")
    print(json.dumps(before_travel_distance, indent=2, ensure_ascii=False))


    # 最適化後のリソースシミュレーション
    after_analyzer = ResourceAnalyzer("../../../データ/test_reorganized.geojson")
    after_total_stats, after_total_resources, after_travel_distance = after_analyzer.run_total_analysis()
    print("\n地域全体の農地統計情報:")
    print(json.dumps(after_total_stats, indent=2, ensure_ascii=False))
    print("\n地域全体の必要な人的リソース:")
    print(json.dumps(after_total_resources, indent=2, ensure_ascii=False))
    print("\n地域全体の移動距離:")
    print(json.dumps(after_travel_distance, indent=2, ensure_ascii=False))



    # # 個々の農家のリソースシミュレーション
    # farmer_id = list(analyzer.farmlands.keys())[0]  # 1つ目の農家のIDを取得
    # farm_stats, resource_requirements = analyzer.run_analysis(farmer_id)
    # print(f"農家ID: {farmer_id}, 農地数: {farm_stats.get('num_plots', 0)}")
    # print("農地統計情報:")
    # print(json.dumps(farm_stats, indent=2, ensure_ascii=False))
    # print("\n必要な人的リソース:")
    # print(json.dumps(resource_requirements, indent=2, ensure_ascii=False))

    # # 地域全体のリソースシミュレーション
    # total_stats, total_resources = analyzer.run_total_analysis()
    # print("\n地域全体の農地統計情報:")
    # print(json.dumps(total_stats, indent=2, ensure_ascii=False))
    # print("\n地域全体の必要な人的リソース:")
    # print(json.dumps(total_resources, indent=2, ensure_ascii=False))


Loaded 116 farmers' farmlands
{'a504f94f9a34e120b11c26dcc561cc93': [<POLYGON ((140.378 36.375, 140.378 36.375, 140.379 36.375, 140.379 36.375, 1...>, <POLYGON ((140.378 36.372, 140.379 36.371, 140.378 36.371, 140.378 36.371, 1...>], '0f1cd9d3f5b6f65893e4f31c96c52045, d0dbbea72d1f340a64f82d1060dc9a42': [<POLYGON ((140.377 36.376, 140.377 36.376, 140.377 36.376, 140.378 36.376, 1...>], '745fb7229d1395c231177495b777fd2e': [<POLYGON ((140.377 36.376, 140.377 36.376, 140.378 36.376, 140.378 36.376, 1...>, <POLYGON ((140.377 36.375, 140.377 36.375, 140.378 36.376, 140.378 36.375, 1...>, <POLYGON ((140.378 36.375, 140.378 36.376, 140.379 36.376, 140.379 36.375, 1...>, <POLYGON ((140.378 36.375, 140.379 36.375, 140.379 36.375, 140.378 36.375, 1...>, <POLYGON ((140.379 36.37, 140.38 36.37, 140.38 36.37, 140.38 36.37, 140.379 ...>], '2e20b8f76a7381f7e923f5f174579807': [<POLYGON ((140.374 36.37, 140.374 36.37, 140.374 36.37, 140.374 36.37, 140.3...>], 'a96dd9b2cfd69e2d52d31a0fee58d768': [<POLYGON

100%|██████████| 116/116 [00:00<00:00, 739.35it/s]



地域全体の農地統計情報:
{
  "total_area": 513551.5166834518,
  "num_plots": 348,
  "num_large_plots": 217
}

地域全体の必要な人的リソース:
51355.15166834516

地域全体の移動距離:
0.0
Loaded 6 farmers' farmlands
{'newfarmer0': [<POLYGON ((140.378 36.375, 140.378 36.375, 140.379 36.375, 140.379 36.375, 1...>, <POLYGON ((140.377 36.376, 140.377 36.376, 140.377 36.376, 140.378 36.376, 1...>, <POLYGON ((140.377 36.376, 140.377 36.376, 140.378 36.376, 140.378 36.376, 1...>, <POLYGON ((140.377 36.375, 140.376 36.375, 140.376 36.375, 140.376 36.375, 1...>, <POLYGON ((140.376 36.376, 140.377 36.376, 140.376 36.376, 140.376 36.376, 1...>, <POLYGON ((140.377 36.376, 140.377 36.375, 140.376 36.375, 140.376 36.375, 1...>, <POLYGON ((140.376 36.375, 140.377 36.375, 140.377 36.375, 140.376 36.375, 1...>, <POLYGON ((140.378 36.375, 140.379 36.375, 140.379 36.375, 140.378 36.375, 1...>, <POLYGON ((140.378 36.376, 140.378 36.376, 140.379 36.376, 140.379 36.376, 1...>, <POLYGON ((140.377 36.375, 140.377 36.375, 140.378 36.376, 140.378 36

100%|██████████| 6/6 [00:00<00:00, 17.24it/s]


地域全体の農地統計情報:
{
  "total_area": 513551.5168763783,
  "num_plots": 348,
  "num_large_plots": 217
}

地域全体の必要な人的リソース:
51355.15168763783

地域全体の移動距離:
0.03765660258999595
